In [ ]:
%%capture
!pip install wandb
!tar -xvf /content/drive/MyDrive/work/Dysarthria_VIVO_system/data/UASpeech.tar

In [ ]:
import wandb
import torch
import os
import pandas as pd
from os import walk
import librosa
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from transformers import AutoFeatureExtractor, AutoModel, HubertModel, Wav2Vec2FeatureExtractor

In [ ]:
# setup wandb
wandb_API_key = "af0ebd78dadd977aadb9b94cc811dc60924219fc"
wandb.login(key = wandb_API_key)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# specify device for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [ ]:
class UASpeechDataset(Dataset):
    def __init__(self, UASpeech_metadata_path, model_path, transform=None, target_transform=None):
        self.metadata = pd.read_csv(UASpeech_metadata_path)

        # Select columns with 'float64' dtype
        float64_cols = list(self.metadata.select_dtypes(include='float64'))

        # The same code again calling the columns
        self.metadata[float64_cols] = self.metadata[float64_cols].astype('float32')

        self.processor = AutoFeatureExtractor.from_pretrained(model_path)

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        audio_path = self.metadata.loc[self.metadata.index[idx], 'path']
        speech, samplerate = sf.read(audio_path)
        # seconds = librosa.get_duration(path= audio_path)
        # print(seconds)
        speech = speech[0:250000]

        preprocessed_speech = self.processor(speech, padding="max_length",  max_length = 250000,return_tensors="pt", sampling_rate = 16000).input_values
        # label = self.metadata.iloc[idx, 3]
        label = self.metadata.loc[self.metadata.index[idx], "Intelligibility_Percentage"]
        return preprocessed_speech, label

In [ ]:
train_set = UASpeechDataset("/content/drive/MyDrive/work/Dysarthria_VIVO_system/data/train_df_reg3.csv", "facebook/hubert-base-ls960")
test_set = UASpeechDataset("/content/drive/MyDrive/work/Dysarthria_VIVO_system/data/test_df_reg3.csv", "facebook/hubert-base-ls960")
val_set = UASpeechDataset("/content/drive/MyDrive/work/Dysarthria_VIVO_system/data/val_df_reg3.csv", "facebook/hubert-base-ls960")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/213 [00:00<?, ?B/s]

In [ ]:
batch_size = 8

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_set, batch_size = batch_size, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size = batch_size, shuffle=True)
val_dataloader = DataLoader(val_set, batch_size = batch_size, shuffle=True)

# Model Creation

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import AutoFeatureExtractor, AutoModel, HubertModel, Wav2Vec2FeatureExtractor
import soundfile as sf

In [ ]:
# Classification Head
class ClassificationHead(nn.Module):
    def __init__(self, hidden_size, num_labels):
        super().__init__()
        self.linearLayer = nn.Linear(hidden_size, hidden_size)
        self.out_proj = nn.Linear(hidden_size, num_labels)

    def forward(self, features):
        x = features
        x = self.linearLayer(x)
        x = torch.tanh(x)
        x = self.out_proj(x)
        return x

In [ ]:
class RegressionHead(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        # self.linearLayer = nn.Linear(hidden_size, hidden_size)
        # self.out_proj = nn.Linear(hidden_size, 1)
        self.seq = nn.Sequential(
              nn.Linear(hidden_size, hidden_size),
              # nn.ReLU(),
              # nn.Linear(hidden_size, hidden_size),
              # nn.ReLU(),
              # nn.Linear(hidden_size, hidden_size),
              # nn.ReLU(),
              # nn.Linear(hidden_size, hidden_size),
              nn.ReLU(),
              nn.Linear(hidden_size, hidden_size),
              nn.ReLU(),
              nn.Linear(hidden_size, 1))

        self.dense = nn.Linear(hidden_size, hidden_size)
        self.dropout = nn.Dropout(0.1)
        self.out_proj = nn.Linear(hidden_size, 1)

    def forward(self, features):
        x = self.seq(features)
        # return x

        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

In [ ]:
# Model class
class Dysarthria_model(nn.Module):
  def __init__(self, model_path = "facebook/hubert-base-ls960", pooling_mode = "mean"):
        super().__init__()
        # self.processor = AutoFeatureExtractor.from_pretrained(model_path)
        self.SSLModel = AutoModel.from_pretrained(model_path)
        self.RegressionHead = RegressionHead(768)
        self.pooling_mode = pooling_mode


  def freeze_feature_extractor(self):
        self.SSLModel.feature_extractor._freeze_parameters()

  def merged_strategy(self, output_features, mode="mean"):
        if mode == "mean":
            outputs = torch.mean(output_features, dim=1)
        elif mode == "sum":
            outputs = torch.sum(output_features, dim=1)
        elif mode == "max":
            outputs = torch.max(output_features, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

  def forward(self, x):


    output_features = self.SSLModel(x).last_hidden_state
    hidden_states = self.merged_strategy(output_features, mode=self.pooling_mode)
    output = self.RegressionHead(hidden_states)

    return output


In [ ]:
model = Dysarthria_model(model_path = "facebook/hubert-base-ls960").to(device)
model.freeze_feature_extractor()

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at facebook/hubert-base-ls960 were not used when initializing HubertModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model fro

In [ ]:
# Test model
preprocessed_speech_sample, label = train_set[1]
output = model(preprocessed_speech_sample.to(device))
print(output.shape)

torch.Size([1, 1])


# Training

In [ ]:
learning_rate = 1e-3

In [ ]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="Dysarthria_regression",
    # id="pwim79rh",
    # resume="must",
    name = "test-run",

    # track hyperparameters and run metadata
    config={
    "learning_rate": learning_rate,
    "architecture": "Hubert for regression",
    "dataset": "UASpeech",
    "epochs": 40,
    }
)

wandb: Currently logged in as: mukhtaralgezoli. Use `wandb login --relogin` to force relogin


In [ ]:
run_id = wandb.run.id
run_name = wandb.run.name
print(run_id)
print(run_name)

zp9717a4
test-run


In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):

    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    model.train()
    train_loss, correct = 0, 0

    for batch, (batch_input, batch_labels) in enumerate(dataloader):
      batch_input = torch.squeeze(batch_input, 1)
      pred = model(batch_input.to(device))
      print("pred -------------------------")
      print(pred)
      print("labels -----------------------")
      print(batch_labels)
      print("/\/\/\/\/\/\/\\//\/\/\/\//\\//\/")
      loss = loss_fn(pred, batch_labels.to(device))
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      train_loss += loss_fn(pred, batch_labels.to(device)).item()
      # correct += (pred.argmax(1) == batch_labels.to(device)).type(torch.float).sum().item()

      if batch % 5 == 0:
            loss, current = loss.item(), (batch + 1) * len(batch_input)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    train_loss /= num_batches
    # correct /= size
    correct = 0

    wandb.log({"train loss": train_loss})
    # wandb.log({"train accuracy": 100*correct})
    return train_loss, 100*correct


def test_loop(dataloader, model, loss_fn):

    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0


    with torch.no_grad():
        for batch_input, batch_labels in dataloader:
            batch_input = torch.squeeze(batch_input, 1)
            pred = model(batch_input.to(device))
            test_loss += loss_fn(pred, batch_labels.to(device)).item()
            # correct += (pred.argmax(1) == batch_labels.to(device)).type(torch.float).sum().item()

    test_loss /= num_batches
    # correct /= size
    correct = 0

    wandb.log({"val loss": test_loss})
    # wandb.log({"val accuracy": 100*correct})

    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")
    # print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    return test_loss, 100*correct


In [ ]:
loss_fn = nn.MSELoss()  # mean square error

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

epochs = 40

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loss, train_acc = train_loop(train_dataloader, model, loss_fn, optimizer)
    val_loss, val_acc = test_loop(val_dataloader, model, loss_fn)
    torch.save({
            'epoch': t,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train loss': train_loss,
            'val loss': val_loss,
            # 'train accuracy': train_acc,
            # 'val accuracy': val_acc,
            "run Id":run_id,
            "run name":run_name,
            }, f"/content/drive/MyDrive/work/Dysarthria_VIVO_system/Saved Models/test_run/Hubert/Checkpoint_reg3_Epoch{t}.pt")

print("Done!")

Epoch 1
-------------------------------
pred -------------------------
tensor([[-0.0269],
        [-0.0569],
        [-0.0879],
        [-0.0660],
        [-0.0886],
        [-0.0141],
        [-0.1518],
        [-0.1251]], device='cuda:0', grad_fn=<AddmmBackward0>)
labels -----------------------
tensor([96.7000, 84.9000, 16.0000,  6.6000, 87.7000, 86.8000, 96.0000,  4.3000])
/\/\/\/\/\/\/\//\/\/\/\//\//\/


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Streaming output truncated to the last 5000 lines.
tensor([[49.4810],
        [49.1295],
        [49.7184],
        [50.2980],
        [50.8150],
        [49.3826],
        [48.7343],
        [48.9811]], device='cuda:0', grad_fn=<AddmmBackward0>)
labels -----------------------
tensor([93.3000,  3.5000, 89.6000, 61.4000, 59.5000, 93.5000, 27.9000, 95.3000])
/\/\/\/\/\/\/\//\/\/\/\//\//\/
pred -------------------------
tensor([[-47.9227],
        [-48.2803],
        [-47.0502],
        [-47.6813],
        [-46.4669],
        [-46.7773],
        [-47.2096],
        [-46.7914]], device='cuda:0', grad_fn=<AddmmBackward0>)
labels -----------------------
tensor([91.5000, 93.8000, 31.4000, 28.7000, 14.7000, 63.8000, 92.3000, 63.6000])
/\/\/\/\/\/\/\//\/\/\/\//\//\/
pred -------------------------
tensor([[48.9916],
        [49.3861],
        [49.8547],
        [48.3621],
        [49.7301],
        [48.7699],
        [50.0334],
        [49.7784]], device='cuda:0', grad_fn=<AddmmBackward0>)
label

KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = test_loop(test_dataloader, model, loss_fn)

In [ ]:
val_loss, val_acc = test_loop(val_dataloader, model, loss_fn)